In [1]:
#input 8-day data
import pandas as pd
import numpy as np
url = "8-day_DATA_US-Ho2_2007_2020_From_1-day.csv" # 8-day intervals

#input parameter
SOS = 16
EOS = 38
Tmax = 40 #℃
Tmin = -1
Topt = 20
e0 = 0.42 # g C /mol PPFD
SOS = 19 # Number of moments in MODIS timeline
EOS = 34

data_train = pd.read_csv(url)
#Converting data types to date types
data_train['Date'] = pd.to_datetime(data_train['Date'])
#Set date to index
df = data_train.set_index('Date')

# Get data for a year like 2010
#print(df['2010'].head())
# Get data for a year-month like 2013-11
#print(df['2013-11'].head())

In [2]:
#Calculated LSWI maximum for growing season (April to November) of each year

# (Not yet finished) It can be updated to "Automatic extraction of start and end years as markers for year-by-year cycle"
Start_Year = 2007
End_Year = 2021
LSWImax_EveryYear = [] # Declare an empty list to store maxinmun of 'GF_LSWI' in growing season(April-Novermber) year by year
for i_year in range(Start_Year,End_Year):

    i_time_start = str(i_year) + '-04' #Convert the data type from int to str in order to retrieve (the retrieved object is a string)
    i_time_end = str(i_year) + '-11' #Convert the data type from int to str in order to retrieve (the retrieved object is a string)
    test = df[i_time_start:i_time_end]['GF_LSWI'].max() #Output maxinmun of 'GF_LSWI' in growing season(April-Novermber) year by year 
    LSWImax_EveryYear.append(test) #Saving values year by year
    
#print(LSWImax_EveryYear)


In [3]:
#Further calculations, excluding noise - Calculated LSWI maximum for growing season (April to November) of each year

#When the model calculation was performed, the LSWImax was not consistent for each year, 
#but considering that the study site is a natural forest with little variation in similar years, 
#the LSWImax for year Yx was taken as the second largest value of the LSWImax from the first two years to the last two years 
#in order to reduce the influence of anomalies received by the LSWImax (e.g., high values due to extreme precipitation, or high values due to winter snow)

#Function# "window_cut": Perform a sliding window operation on a list and select the data in the window
#data_list: the original data column. 
#windows_size: the size of the window.  
#step: step length of the window.
def window_cut(data_list, window_size, step):
    data_len = len(data_list)
    cut_num = int((data_len - window_size) / step)
    res = []
    for i in range(cut_num+1):
        start_index = i  * step
        end_index = start_index + window_size
        cur_data = data_list[start_index:end_index]
        #print(cur_data) #Output the list inside each window
        res.append(cur_data)        
    return res

window_size = 5
step = 1
LSWImax_EveryYear_in_window = window_cut(LSWImax_EveryYear, window_size, step)
#print(window_cut(LSWImax_EveryYear, window_size, step))

Second_LSWImax_Every_5_Year = []
for i in range(len(LSWImax_EveryYear_in_window)):
    tempo = LSWImax_EveryYear_in_window[i]
    tempo = np.array(tempo)
    max2 = np.sort(tempo)[-2]
    Second_LSWImax_Every_5_Year.append(max2) #Second_LSWImax_Every_5_Year contain 2009-2018

Second_LSWImax_Every_5_Year_all = [] #Second_LSWImax_Every_5_Year_all contain 2007-2020
Second_LSWImax_Every_5_Year_all.append(Second_LSWImax_Every_5_Year[0])
Second_LSWImax_Every_5_Year_all.append(Second_LSWImax_Every_5_Year[0])
for i in range(len(Second_LSWImax_Every_5_Year)):
    Second_LSWImax_Every_5_Year_all.append(Second_LSWImax_Every_5_Year[i])
Second_LSWImax_Every_5_Year_all.append(Second_LSWImax_Every_5_Year[len(Second_LSWImax_Every_5_Year)-1])
Second_LSWImax_Every_5_Year_all.append(Second_LSWImax_Every_5_Year[len(Second_LSWImax_Every_5_Year)-1])


In [4]:
#Calculate Wscalar

#Assign the calculated LSWImax per year to an array of 8-day intervals
Second_LSWImax_Every_5_Year_8_interval = []
for i in range(len(Second_LSWImax_Every_5_Year_all)):
    for j in range(46):
        Second_LSWImax_Every_5_Year_8_interval.append(Second_LSWImax_Every_5_Year_all[i])

data_train['Second_LSWImax_Every_5_Year_8_interval'] = Second_LSWImax_Every_5_Year_8_interval

Second_LSWImax_Every_5_Year_8_interval = data_train['Second_LSWImax_Every_5_Year_8_interval'] 
LSWI = data_train['GF_LSWI']

Wscalar = (1 + LSWI) / (1 + Second_LSWImax_Every_5_Year_8_interval)
data_train['Wscalar_Calculate'] = Wscalar


In [5]:
#Calculate Tscalar

#Assign the calculated LSWImax per year to an array of 8-day intervals

Tec = data_train['TA_8D']

T_Abstract_Tmin = Tec - Tmin;
T_Abstract_Tmax = Tec - Tmax;
T_Abstract_Topt = Tec - Topt;
Tscalar = (T_Abstract_Tmin * T_Abstract_Tmax) / ((T_Abstract_Tmin * T_Abstract_Tmax) - (T_Abstract_Topt * T_Abstract_Topt)) 

Tscalar[Tscalar<0] = 0 #Should have taken Tec < Tmin Tscalar is 0, here directly take Tscalar (Tscalar < 0) = 0

data_train['Tscalar_Calculate'] = Tscalar
print(data_train['Tscalar_Calculate'])

0      0.324079
1      0.000000
2      0.000000
3      0.000000
4      0.000000
         ...   
639    0.104428
640    0.385632
641    0.000000
642    0.000000
643    0.116576
Name: Tscalar_Calculate, Length: 644, dtype: float64


In [6]:
#Calculate FPAR

EVI = data_train['GF_EVI']
FPAR_PAV = (EVI - 0.1) * 1.25
data_train['FPAR_PAV_Calculate'] = FPAR_PAV


In [7]:
#Calculate eg

eg =  e0 * Tscalar * Wscalar
data_train['eg_Calculate'] = eg


In [8]:
#Calculate GPPvpm

PAR = data_train['PAR_8D']
GPPvpm = eg * FPAR_PAV * PAR
GPPvpm[GPPvpm<0] = 0
data_train['GPPvpm_Calculate'] = GPPvpm
print(data_train['GPPvpm_Calculate'])


0           NaN
1      0.000000
2      0.000000
3      0.000000
4      0.000000
         ...   
639    0.079089
640    0.319948
641    0.000000
642    0.000000
643    0.112510
Name: GPPvpm_Calculate, Length: 644, dtype: float64


In [9]:
data_train.to_csv("8-day_DATA_US-Ho2_2007_2020_ForPY_Calculate.csv",index=False,sep=',')